## Descripcion

Este codigo consiste en realizar un ETL de los datos de la carpeta 'ignore':  
1-Extraccion: Importa 4 datasets correspondientes a los titulos por plataforma y 8 datasets correspondientes a las puntuaciones de titulos por usuario.  
2-Transformacion: se realizan 2 transformaciones  
    a) All: Busca unificar los titulos de las 4 plataformas para unificar las consultas en un dataset 'All'.  
    b) Score: Busca unificar las puntuaciones de titulos para consultas de score en un unico dataset 'Score'.   
3-Carga: Exporta los dataframes a los siguientes archivos:  
    a) All => all.csv  
    b) Score => score.csv  
    c) Score => score.parquet  
    d) Cast => cast.csv  

## Extraccion

Importa 4 datasets correspondientes a los titulos por plataforma y 8 datasets correspondientes a las puntuaciones de titulos por usuario.

In [35]:
import pandas as pd
import numpy as np

In [36]:
# Titles
Amazon = pd.read_csv("ignore/amazon_prime_titles.csv")
Disney = pd.read_csv("ignore/disney_plus_titles.csv")
Hulu = pd.read_csv("ignore/hulu_titles.csv")
Netflix = pd.read_csv("ignore/netflix_titles.csv")

In [ ]:
# Scores
a = pd.read_csv("ignore/ratings/1.csv")
b = pd.read_csv("ignore/ratings/2.csv")
c = pd.read_csv("ignore/ratings/3.csv")
d = pd.read_csv("ignore/ratings/4.csv")
e = pd.read_csv("ignore/ratings/5.csv")
f = pd.read_csv("ignore/ratings/6.csv")
g = pd.read_csv("ignore/ratings/7.csv")
h = pd.read_csv("ignore/ratings/8.csv")

In [ ]:
print('Amazon')
print('Duplicates:',Amazon.duplicated().any())
print('Files, Columns:',Amazon.shape)
print(Amazon.isnull().sum())
Amazon.tail(3)

In [ ]:
print('Disney')
print('Duplicates:',Disney.duplicated().any())
print('Files, Columns:',Disney.shape)
print(Disney.isnull().sum())
Disney.tail(3)

In [ ]:
print('Hulu')
print('Duplicates:',Hulu.duplicated().any())
print('Files, Columns:',Hulu.shape)
print(Hulu.isnull().sum())
Hulu.tail(3)

In [ ]:
print('Netflix')
print('Duplicates:',Netflix.duplicated().any())
print('Files, Columns:',Netflix.shape)
print(Netflix.isnull().sum())
Netflix.tail(3)

## Transformacion All

In [37]:
# Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, 
# seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)
id_A = 'a' + Amazon['show_id']
Amazon.insert(0, 'id', id_A)
Amazon.insert(1, 'platform', 'amazon')

id_D = 'd' + Disney['show_id']
Disney.insert(0, 'id', id_D)
Disney.insert(1, 'platform', 'disney')

id_H = 'h' + Hulu['show_id']
Hulu.insert(0, 'id', id_H)
Hulu.insert(1, 'platform', 'hulu')

id_N = 'n' + Netflix['show_id']
Netflix.insert(0, 'id', id_N)
Netflix.insert(1, 'platform', 'netflix')

In [38]:
All = pd.concat([Amazon,Disney,Hulu,Netflix], axis=0).reset_index()
print('All platforms')
print('Files, Columns:',All.shape)
print(All.isnull().sum())
All.head(3)


All platforms
Files, Columns: (22998, 15)
index               0
id                  0
platform            0
show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
dtype: int64


,index,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,as1,amazon,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,1,as2,amazon,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,2,as3,amazon,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [39]:
# Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”)
All['rating'].fillna('G',inplace=True)
print(All['rating'].isnull().sum())

0


In [40]:
# De haber fechas, deberán tener el formato AAAA-mm-dd
All['date_added'] = pd.to_datetime(All['date_added'])
All.tail(3)

,index,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
22995,8804,ns8805,netflix,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
22996,8805,ns8806,netflix,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
22997,8806,ns8807,netflix,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,2019-03-02,2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [41]:
# Los campos de texto deberán estar en minúsculas, sin excepciones
lowercase = lambda s:s.lower() if type(s) == str else s
All = All.applymap(lowercase)

In [42]:
# El campo duration debe convertirse en dos campos: duration_int y duration_type. 
# El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)
All[['duration_int','duration_type']] = All['duration'].str.split(' ', expand=True)
All.drop(columns=['duration'], inplace=True)
All.nunique()          

index             9668
id               22998
platform             4
show_id           9668
type                 2
title            22042
director         10095
cast             16744
country            886
date_added        2003
release_year       101
rating             105
listed_in         1687
description      22669
duration_int       225
duration_type        3
dtype: int64

In [43]:
# Organiza las columnas
All = All.reindex(columns=['id', 'platform', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'duration_int', 'duration_type', 'listed_in', 'description'])
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22516 non-null  object        
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 2.3+ MB


In [ ]:
# Contabiliza valores nulos
All.isnull().sum()

In [ ]:
# Cantidad de valores unicos
All.nunique()

In [44]:
# Convierte a integer los valores numericos
All['duration_int'] = All['duration_int'].fillna(0)
All['duration_int'] = All['duration_int'].astype(int)
All['release_year'] = All['release_year'].astype(int) 
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22998 non-null  int64         
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 2.3+ MB


In [ ]:
All.nunique()

In [45]:
# VIsualiza que categorias de datos hay en duration_type
import collections
collections.Counter(All['duration_type'])

Counter({'min': 15999, 'season': 4183, 'seasons': 2334, nan: 482})

In [46]:
# Unifica seasons y season
All['duration_type'] = All['duration_type'].replace('seasons','season')
collections.Counter(All['duration_type'])

Counter({'min': 15999, 'season': 6517, nan: 482})

In [47]:
# Lista los encabezados
headers = list(All.columns)
print(headers)  

['id', 'platform', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'duration_int', 'duration_type', 'listed_in', 'description']


In [48]:
# Organiza y reduce las columnas
All = All.reindex(columns=['id', 'platform', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'duration_int', 'duration_type', 'listed_in', 'description'])
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22998 non-null  int64         
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 2.3+ MB


In [49]:
# Opcion de guardar avance parcial cambiando switch a True (solo con carpeta ignore)
switch = False
if switch:
    All.to_csv('ignore/all_titles.csv', index=False)

## Transformacion Score

In [ ]:
# Unifica los dataset en Score
Score = pd.concat([a,b,c,d,e,f,g,h], axis=0).reset_index()
print('Score')
print('Files, Columns:',Score.shape)
# print(Score.isnull().sum())
Score.tail(3)

In [ ]:
# Agrega las columnas year y platform, renombra rating a score
Score = Score.rename(columns={'rating':'score'})
Score['year'] = pd.to_datetime(Score['timestamp'], unit='s').dt.year
Score['platform'] = Score['movieId'].str[0]
# Score = Score[['platform','score','year','movieId']]
Score.tail(3)

In [ ]:
# Completa el nombre de las plataformas mediante una funcion
def get_initial(movieId:str):
    if movieId[0] == 'a':
        platform = 'amazon'
    elif movieId[0] == 'd':
        platform = 'disney'
    elif movieId[0] == 'h':
        platform = 'hulu'
    elif movieId[0] == 'n':
        platform = 'netflix'
    else: platform = '' 
    
    return platform

get_initial('as883')

# Aplicar la función a la columna 'nombre' para crear la columna 'inicial'
Score['platform'] = Score['platform'].apply(get_initial)
Score.drop(columns='index',inplace=True)
Score.head(3)

In [ ]:
# Reduce la cantidad de columnas
Score = Score[['userId','score','movieId','year','platform']]
Score

In [ ]:
# Definir una función de agregación que calcula los promedios del score y consulta valores estadisticos
S = Score.groupby('movieId').agg(pd.Series({'score': 'mean'}))
S.describe()

## Carga

In [50]:
# Opcion de cargar avance parcial cambiando switch a True (solo con carpeta ignore)
switch = False
if switch:
    All = pd.read_csv('ignore/all_titles.csv', index=False)

In [51]:
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22998 non-null  int64         
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 2.3+ MB


In [ ]:
Score.info()

In [ ]:
All['release_year'].max()
All['duration_int'].max()
Score['userId'].max()

In [ ]:
All['platform'] = All['platform'].astype(str)
All['type'] = All['type'].astype(str)
All['title'] = All['title'].astype(str)
All['director'] = All['director'].astype(str)
All['cast'] = All['cast'].astype(str)
All['country'] = All['country'].astype(str)
All['type'] = All['type'].astype(str)
All['title'] = All['title'].astype(str)
All['director'] = All['director'].astype(str)
All['release_year'] = All['release_year'].astype(np.int16)
All['duration_int'] = All['duration_int'].astype(np.int16)
All['listed_in'] = All['listed_in'].astype(str)
All['description'] = All['description'].astype(str)

Score['userId'] = Score['userId'].astype(np.int32)
Score['score'] = Score['score'].astype(np.int8)
Score['platform'] = Score['platform'].astype(str)

In [ ]:
Score.head()

In [ ]:
All.to_csv('data/all.csv', index=False)

In [ ]:
# Opcion de exportar csv de gran tamanio cambiando switch a True (solo con carpeta ignore)
switch = False
if switch:
    Score.to_csv('ignore/score.csv', index=False) 
# Da un archivo de 300Mb

In [ ]:
# S.to_csv('data/score_mean.csv', index=False)

In [ ]:
# Convierte score.csv en score.parquet para reducir el tamanio del archivo
# Score = pd.read_csv('ignore/score.csv')
Score.to_parquet('data/score.parquet')